<a href="https://colab.research.google.com/github/abkerper/project_voting/blob/main/Connor%202008-2020%20Merged%20Dataframes.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
! git clone https://github.com/DS3001/project_voting
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
from sklearn import linear_model
from sklearn.model_selection import train_test_split

Cloning into 'project_voting'...
remote: Enumerating objects: 70, done.
remote: Counting objects: 100% (8/8), done.
remote: Compressing objects: 100% (5/5), done.
remote: Total 70 (delta 5), reused 3 (delta 3), pack-reused 62
Receiving objects: 100% (70/70), 31.37 MiB | 10.85 MiB/s, done.
Resolving deltas: 100% (29/29), done.
Updating files: 100% (41/41), done.


In [2]:
data_df = pd.read_csv("./project_voting/data/voting_VA.csv")
data_df.head()

,Unnamed: 0,year,state,state_po,county_name,county_fips,office,candidate,party,candidatevotes,totalvotes,version,mode
0,11161,2000,VIRGINIA,VA,ACCOMACK,51001,US PRESIDENT,AL GORE,DEMOCRAT,5092,11925,20220315,TOTAL
1,11162,2000,VIRGINIA,VA,ACCOMACK,51001,US PRESIDENT,GEORGE W. BUSH,REPUBLICAN,6352,11925,20220315,TOTAL
2,11163,2000,VIRGINIA,VA,ACCOMACK,51001,US PRESIDENT,RALPH NADER,GREEN,220,11925,20220315,TOTAL
3,11164,2000,VIRGINIA,VA,ACCOMACK,51001,US PRESIDENT,OTHER,OTHER,261,11925,20220315,TOTAL
4,11165,2000,VIRGINIA,VA,ALBEMARLE,51003,US PRESIDENT,AL GORE,DEMOCRAT,16255,36846,20220315,TOTAL


In [3]:
data_2000 = data_df[(data_df['year'] == 2000)]
data_2004 = data_df[(data_df['year'] == 2004)]
data_2008 = data_df[(data_df['year'] == 2008)]
data_2012 = data_df[(data_df['year'] == 2012)]
data_2016 = data_df[(data_df['year'] == 2016)]
data_2020 = data_df[(data_df['year'] == 2020)]
data_2008.head()

,Unnamed: 0,year,state,state_po,county_name,county_fips,office,candidate,party,candidatevotes,totalvotes,version,mode
938,30189,2008,VIRGINIA,VA,ACCOMACK,51001,US PRESIDENT,BARACK OBAMA,DEMOCRAT,7607,15623,20220315,TOTAL
939,30190,2008,VIRGINIA,VA,ACCOMACK,51001,US PRESIDENT,JOHN MCCAIN,REPUBLICAN,7833,15623,20220315,TOTAL
940,30191,2008,VIRGINIA,VA,ACCOMACK,51001,US PRESIDENT,OTHER,OTHER,183,15623,20220315,TOTAL
941,30192,2008,VIRGINIA,VA,ALBEMARLE,51003,US PRESIDENT,BARACK OBAMA,DEMOCRAT,29792,50984,20220315,TOTAL
942,30193,2008,VIRGINIA,VA,ALBEMARLE,51003,US PRESIDENT,JOHN MCCAIN,REPUBLICAN,20576,50984,20220315,TOTAL


In [4]:
demo_2008 = pd.read_csv("/content/project_voting/data/county_data/0002_ds176_20105_county_E.csv", encoding='latin1')
#This is the dataset that is a monster

row = demo_2008.iloc[0]  # Row 0 contains the descriptions of the variables

# Prints out each cell to show variables but honestly easier to just look at the codebook
print("Variables in Dataset:")

Variables in Dataset:


<ipython-input-4-f1f58fc645a4>:1: DtypeWarning: Columns (3,4,6,8,9,10,11,12,13,14,15,16,17,18,19,20,21,22,23,24,25,26,27,28,29,30,31,32,33,34,35,38,39,40,41,42,43,44,45,46,47,48,49,50,51,52,53,54,55,56,57,58,59,60,61,62,63,64,65,66,67,68,69,70,71,72,73,74,75,76,77,78,79,80,81,82,83,84,85,86,87,88,89,90,91,92,93,94,95,96,97,98,99,100,101,102,103,104,105,106,107,108,109,110,111,112,113,114,115,116,117,118,119,120,121,122,123,124,125,126,127,128,129,130,131,132,133,134,135,136,137,138,139,140,141,142,143,144,145,146,147,148,149,150,151,152,153,154,155,156,157,158,159,160,161,162,163,164,165,166,167,168,169,170,171,172,173,174,175,176,177,178,179,180,181,182,183,184,185,186,187,188,189,190,191,192,193,194,195,196,197,198,199,200,201,202,203,204,205,206,207,208,209,210,211,212,213,214,215,216,217,218,219,220,221,222,223,224,225,226,227,228,229,230,231,232,233,234,235,236,237,238,239,240,241,242,243,244,245,246,247,248,249,250,251,253,254,255,256,257,258,259,260,261,262,263,264,265,266,267,2

In [5]:
demo_2008 = demo_2008[demo_2008["STUSAB"] == "VA"] # Sorts the df for just VA counties. Can change if we want to look at US as a whole
demo_2008.head()

demo_2008= demo_2008.loc[:,("COUNTY","JMAE001","JMBE002","JMBE003","JMBE004","JMBE005",
                     "JMBE006","JMBE007","JMBE008","JQBE001","JL0E001","JL0E002",
                     "JL0E003")]  #Selects variables we want in the clean df

new_column_names = {"JMAE001" : "Total Population",
        "JMBE002":     "White alone",
        "JMBE003":    "Black or African American alone",
        "JMBE004":     "American Indian and Alaska Native alone",
        "JMBE005":     "Asian alone",
        "JMBE006":     "Native Hawaiian and Other Pacific Islander alone",
        "JMBE007":     "Some other race alone",
        "JMBE008":     "Two or more races",
        "JQBE001":     "Per Capita income in the past 12 months",
        "JL0E001":     "Median age: Total",
        "JL0E002":     "Median age: Male",
        "JL0E003":     "Median age: Female",
}  #These are all taken from the codebook in the github. MAKE SURE YOU'RE LOOKING AT THE RIGHT CODEBOOK FOR THE RIGHT CSV. I didnt realize that until way too late.

demo_2008.rename(columns=new_column_names, inplace=True)
demo_2008.head()

,COUNTY,Total Population,White alone,Black or African American alone,American Indian and Alaska Native alone,Asian alone,Native Hawaiian and Other Pacific Islander alone,Some other race alone,Two or more races,Per Capita income in the past 12 months,Median age: Total,Median age: Male,Median age: Female
2821,Accomack County,34066,22639,9729,30,166,0,702,800,22766.0,43.9,41.5,45.8
2822,Albemarle County,96633,79493,9563,168,4146,28,1164,2071,36685.0,37.6,36.3,38.9
2823,Alleghany County,16406,15250,979,3,81,0,18,75,22013.0,44.7,43.2,46.0
2824,Amelia County,12517,9228,2954,31,47,18,125,114,24197.0,41.5,40.6,42.7
2825,Amherst County,32315,24762,6201,301,149,0,310,592,21097.0,41.1,39.9,42.2


In [6]:
data_by_year = [data_2000, data_2004, data_2008, data_2012, data_2016, data_2020]
df_wins = pd.DataFrame()

for year in data_by_year:
    wins = pd.DataFrame()
    county_names = year['county_name'].values
    county_names = list(set(county_names))
    for county in county_names:
        county_rows = year[year['county_name'] == county]
        max_votes = county_rows['candidatevotes'].idxmax()
        winning_row = year.loc[max_votes]
        wins = pd.concat([wins, pd.DataFrame([winning_row])], ignore_index=True)
    df_wins = pd.concat([df_wins, wins], ignore_index=True)

data_2000 = df_wins[df_wins['year'] == 2000]
data_2004 = df_wins[df_wins['year'] == 2004]
data_2008 = df_wins[df_wins['year'] == 2008]
data_2012 = df_wins[df_wins['year'] == 2012]
data_2016 = df_wins[df_wins['year'] == 2016]
data_2020 = df_wins[df_wins['year'] == 2020]

In [8]:
print(type(data_2008))


<class 'pandas.core.frame.DataFrame'>


#CODE FOR 2008:

In [13]:
data_2008 = data_2008.rename(columns={'county_name': 'COUNTY'})
data_2008['COUNTY'] = data_2008['COUNTY'].str.lower()
data_2008.head()

demo_2008['COUNTY'] = demo_2008['COUNTY'].str.lower()
demo_2008.head()

demo_2008['COUNTY'] = demo_2008['COUNTY'].str.replace(' ', '')
demo_2008['COUNTY'] = demo_2008['COUNTY'].str.replace('county', '')
data_2008['COUNTY'] = data_2008['COUNTY'].str.replace(' ', '')

demo_2008['COUNTY'] = demo_2008['COUNTY'].str.strip()
data_2008['COUNTY'] = data_2008['COUNTY'].str.strip()
merged_df_2008 = pd.merge(data_2008, demo_2008, on="COUNTY")
merged_df_2008.head()


,Unnamed: 0,year,state,state_po,COUNTY,county_fips,office,candidate,party,candidatevotes,...,Black or African American alone,American Indian and Alaska Native alone,Asian alone,Native Hawaiian and Other Pacific Islander alone,Some other race alone,Two or more races,Per Capita income in the past 12 months,Median age: Total,Median age: Male,Median age: Female
0,30412,2008,VIRGINIA,VA,rappahannock,51157,US PRESIDENT,JOHN MCCAIN,REPUBLICAN,2227,...,449,31,12,0,47,110,37149.0,46.8,47.0,46.8
1,30345,2008,VIRGINIA,VA,loudoun,51107,US PRESIDENT,BARACK OBAMA,DEMOCRAT,74845,...,20895,392,40992,189,12134,9338,45356.0,34.3,33.9,34.6
2,30207,2008,VIRGINIA,VA,arlington,51013,US PRESIDENT,BARACK OBAMA,DEMOCRAT,78994,...,16871,700,18768,158,15100,7291,57724.0,33.9,34.0,33.8
3,30220,2008,VIRGINIA,VA,bland,51021,US PRESIDENT,JOHN MCCAIN,REPUBLICAN,2031,...,198,0,94,0,7,16,20468.0,44.6,44.7,44.4
4,30202,2008,VIRGINIA,VA,amherst,51009,US PRESIDENT,JOHN MCCAIN,REPUBLICAN,8470,...,6201,301,149,0,310,592,21097.0,41.1,39.9,42.2


# CODE FOR 2012:

In [14]:
demo_2012 = pd.read_csv("/content/project_voting/data/county_data/0002_ds225_20165_county_E.csv", encoding='latin1')
#This is the dataset that is a monster

row = demo_2012.iloc[0]  # Row 0 contains the descriptions of the variables

# Prints out each cell to show variables but honestly easier to just look at the codebook
print("Variables in Dataset:")

Variables in Dataset:


In [15]:
demo_2012 = demo_2012[demo_2012["STUSAB"] == "VA"] # Sorts the df for just VA counties. Can change if we want to look at US as a whole
demo_2012.head()

demo_2012= demo_2012.loc[:,("COUNTY","AF2AE001","AF2ME002","AF2ME003","AF2ME004","AF2ME005",
                     "AF2ME006","AF2ME007","AF2ME008","AF6AE001","AF2AE001","AF2AE002",
                     "AF2AE026")]  #Selects variables we want in the clean df

new_column_names = {"AF2AE001" : "Total Population",
        "AF2ME002":     "White alone",
        "AF2ME003":    "Black or African American alone",
        "AF2ME004":     "American Indian and Alaska Native alone",
        "AF2ME005":     "Asian alone",
        "AF2ME006":     "Native Hawaiian and Other Pacific Islander alone",
        "AF2ME007":     "Some other race alone",
        "AF2ME008":     "Two or more races",
        "AF6AE001":     "Per Capita income in the past 12 months",
        "AF2AE001":     "Median age: Total",
        "AF2AE002":     "Median age: Male",
        "AF2AE026":     "Median age: Female",
}  #These are all taken from the codebook in the github. MAKE SURE YOU'RE LOOKING AT THE RIGHT CODEBOOK FOR THE RIGHT CSV. I didnt realize that until way too late.

demo_2012.rename(columns=new_column_names, inplace=True)
demo_2012.head()

,COUNTY,Median age: Total,White alone,Black or African American alone,American Indian and Alaska Native alone,Asian alone,Native Hawaiian and Other Pacific Islander alone,Some other race alone,Two or more races,Per Capita income in the past 12 months,Median age: Total,Median age: Male,Median age: Female
2821,Accomack County,33060,22369,9573,122,62,0,488,446,23337,33060,16125,16935
2822,Albemarle County,104287,84955,9747,250,4733,34,1627,2941,38039,104287,49946,54341
2823,Alleghany County,15919,14745,829,2,23,6,68,246,25220,15919,7788,8131
2824,Amelia County,12793,9389,3010,65,16,0,33,280,25335,12793,6642,6151
2825,Amherst County,31999,24609,5717,117,185,14,267,1090,23372,31999,15346,16653


In [16]:
data_2012 = data_2012.rename(columns={'county_name': 'COUNTY'})
data_2012['COUNTY'] = data_2012['COUNTY'].str.lower()
data_2012.head()

demo_2012['COUNTY'] = demo_2012['COUNTY'].str.lower()
demo_2012.head()

demo_2012['COUNTY'] = demo_2012['COUNTY'].str.replace(' ', '')
demo_2012['COUNTY'] = demo_2012['COUNTY'].str.replace('county', '')
data_2012['COUNTY'] = data_2012['COUNTY'].str.replace(' ', '')

demo_2012['COUNTY'] = demo_2012['COUNTY'].str.strip()
data_2012['COUNTY'] = data_2012['COUNTY'].str.strip()
merged_df_2012 = pd.merge(data_2012, demo_2012, on="COUNTY")
merged_df_2012.head()

,Unnamed: 0,year,state,state_po,COUNTY,county_fips,office,candidate,party,candidatevotes,...,Black or African American alone,American Indian and Alaska Native alone,Asian alone,Native Hawaiian and Other Pacific Islander alone,Some other race alone,Two or more races,Per Capita income in the past 12 months,Median age: Total,Median age: Male,Median age: Female
0,39763,2012,VIRGINIA,VA,rappahannock,51157,US PRESIDENT,MITT ROMNEY,REPUBLICAN,2311,...,311,14,48,0,19,188,34886,7416,3643,3773
1,39696,2012,VIRGINIA,VA,loudoun,51107,US PRESIDENT,BARACK OBAMA,DEMOCRAT,82479,...,26600,841,60404,325,12586,17601,48578,362435,179315,183120
2,39558,2012,VIRGINIA,VA,arlington,51013,US PRESIDENT,BARACK OBAMA,DEMOCRAT,81269,...,19477,883,22655,195,12744,6957,64746,226092,112644,113448
3,39571,2012,VIRGINIA,VA,bland,51021,US PRESIDENT,MITT ROMNEY,REPUBLICAN,2144,...,286,0,3,4,0,36,22249,6612,3733,2879
4,39553,2012,VIRGINIA,VA,amherst,51009,US PRESIDENT,MITT ROMNEY,REPUBLICAN,8876,...,5717,117,185,14,267,1090,23372,31999,15346,16653


#CODE FOR 2016:

In [17]:
data_2016 = data_2016.rename(columns={'county_name': 'COUNTY'})
data_2016['COUNTY'] = data_2016['COUNTY'].str.lower()
data_2016.head()

demo_2012['COUNTY'] = demo_2012['COUNTY'].str.lower()
demo_2012.head()

demo_2012['COUNTY'] = demo_2012['COUNTY'].str.replace(' ', '')
demo_2012['COUNTY'] = demo_2012['COUNTY'].str.replace('county', '')
data_2016['COUNTY'] = data_2016['COUNTY'].str.replace(' ', '')

demo_2012['COUNTY'] = demo_2012['COUNTY'].str.strip()
data_2016['COUNTY'] = data_2016['COUNTY'].str.strip()
merged_df_2016 = pd.merge(data_2016, demo_2012, on="COUNTY")
merged_df_2016.head()

,Unnamed: 0,year,state,state_po,COUNTY,county_fips,office,candidate,party,candidatevotes,...,Black or African American alone,American Indian and Alaska Native alone,Asian alone,Native Hawaiian and Other Pacific Islander alone,Some other race alone,Two or more races,Per Capita income in the past 12 months,Median age: Total,Median age: Male,Median age: Female
0,49114,2016,VIRGINIA,VA,rappahannock,51157,US PRESIDENT,DONALD TRUMP,REPUBLICAN,2539,...,311,14,48,0,19,188,34886,7416,3643,3773
1,49047,2016,VIRGINIA,VA,loudoun,51107,US PRESIDENT,HILLARY CLINTON,DEMOCRAT,100795,...,26600,841,60404,325,12586,17601,48578,362435,179315,183120
2,48909,2016,VIRGINIA,VA,arlington,51013,US PRESIDENT,HILLARY CLINTON,DEMOCRAT,92016,...,19477,883,22655,195,12744,6957,64746,226092,112644,113448
3,48922,2016,VIRGINIA,VA,bland,51021,US PRESIDENT,DONALD TRUMP,REPUBLICAN,2573,...,286,0,3,4,0,36,22249,6612,3733,2879
4,48904,2016,VIRGINIA,VA,amherst,51009,US PRESIDENT,DONALD TRUMP,REPUBLICAN,9719,...,5717,117,185,14,267,1090,23372,31999,15346,16653


Used the 2012-2016 data set for both 2012 and 2016

#CODE FOR 2020:

In [19]:
demo_2020 = pd.read_csv("/content/project_voting/data/county_data/0002_ds249_20205_county_E.csv", encoding='latin1')

row = demo_2012.iloc[0]  # Row 0 contains the descriptions of the variables

# Prints out each cell to show variables but honestly easier to just look at the codebook
print("Variables in Dataset:")

<ipython-input-19-1dac20a4122d>:1: DtypeWarning: Columns (3,4,6,8,9,10,11,12,13,14,15,16,17,18,19,20,21,22,23,24,25,26,27,28,29,30,31,32,33,34,35,36,39,40,41,42,43,44,45,46,47,48,49,50,51,52,53,54,55,56,57,58,59,60,61,62,63,64,65,66,67,68,69,70,71,72,73,74,75,76,77,78,79,80,81,82,83,84,85,86,87,88,89,90,91,92,93,94,95,96,97,98,99,100,101,102,103,104,105,106,107,108,109,110,111,112,113,114,115,116,117,118,119,120,121,122,123,124,125,126,127,128,129,130,131,132,133,134,135,136,137,138,139,140,141,142,143,144,145,146,147,148,149,150,151,152,153,154,155,156,157,158,159,160,161,162,163,164,165,166,167,168,169,170,171,172,173,174,175,176,177,178,179,180,181,182,183,184,185,186,187,188,189,190,191,192,193,194,195,196,197,198,199,200,201,202,203,204,205,206,207,208,209,210,211,212,213,214,215,216,217,218,219,220,221,222,223,224,225,226,227,228,229,230,231,232,233,234,235,236,237,238,239,240,241,242,243,244,245,246,247,248,249,250,251,252,253,254,255,256,257,258,259,260,261,262,263,264,265,266,

Variables in Dataset:


In [21]:
demo_2020 = demo_2020[demo_2020["STUSAB"] == "VA"] # Sorts the df for just VA counties. Can change if we want to look at US as a whole
demo_2020.head()

demo_2020= demo_2020.loc[:,("COUNTY","AMPWE001","AMPWE002","AMPWE003","AMPWE004","AMPWE005",
                     "AMPWE006","AMPWE007","AMPWE008","AMTCE001","AMPLE001","AMPLE002",
                     "AMPLE003")]  #Selects variables we want in the clean df

new_column_names = {"AMPWE001" : "Total Population",
        "AMPWE002":     "White alone",
        "AMPWE003":    "Black or African American alone",
        "AMPWE004":     "American Indian and Alaska Native alone",
        "AMPWE005":     "Asian alone",
        "AMPWE006":     "Native Hawaiian and Other Pacific Islander alone",
        "AMPWE007":     "Some other race alone",
        "AMPWE008":     "Two or more races",
        "AMTCE001":     "Per Capita income in the past 12 months",
        "AMPLE001":     "Median age: Total",
        "AMPLE002":     "Median age: Male",
        "AMPLE003":     "Median age: Female",
}  #These are all taken from the codebook in the github. MAKE SURE YOU'RE LOOKING AT THE RIGHT CODEBOOK FOR THE RIGHT CSV. I didnt realize that until way too late.

demo_2020.rename(columns=new_column_names, inplace=True)
demo_2020.head()

,COUNTY,Total Population,White alone,Black or African American alone,American Indian and Alaska Native alone,Asian alone,Native Hawaiian and Other Pacific Islander alone,Some other race alone,Two or more races,Per Capita income in the past 12 months,Median age: Total,Median age: Male,Median age: Female
2822,Accomack County,32560,21773,9390,48,244,0,154,951,27012,45.8,42.6,47.1
2823,Albemarle County,108819,87871,10005,315,5981,48,989,3610,46241,39.4,37.7,40.7
2824,Alleghany County,15030,13925,790,0,10,0,64,241,28048,48.0,46.1,50.0
2825,Amelia County,12970,9789,2876,0,1,0,0,304,32799,45.0,44.4,48.6
2826,Amherst County,31782,24330,5758,243,209,12,203,1027,28866,44.9,42.0,46.0


In [22]:
data_2020= data_2020.rename(columns={'county_name': 'COUNTY'})
data_2020['COUNTY'] = data_2020['COUNTY'].str.lower()
data_2020.head()

demo_2020['COUNTY'] = demo_2020['COUNTY'].str.lower()
demo_2020.head()

demo_2020['COUNTY'] = demo_2020['COUNTY'].str.replace(' ', '')
demo_2020['COUNTY'] = demo_2020['COUNTY'].str.replace('county', '')
data_2020['COUNTY'] = data_2020['COUNTY'].str.replace(' ', '')

demo_2020['COUNTY'] = demo_2020['COUNTY'].str.strip()
data_2020['COUNTY'] = data_2020['COUNTY'].str.strip()
merged_df_2020 = pd.merge(data_2020, demo_2020, on="COUNTY")
merged_df_2020.head()

,Unnamed: 0,year,state,state_po,COUNTY,county_fips,office,candidate,party,candidatevotes,...,Black or African American alone,American Indian and Alaska Native alone,Asian alone,Native Hawaiian and Other Pacific Islander alone,Some other race alone,Two or more races,Per Capita income in the past 12 months,Median age: Total,Median age: Male,Median age: Female
0,71277,2020,VIRGINIA,VA,rappahannock,51157,US PRESIDENT,DONALD J TRUMP,REPUBLICAN,2155,...,334,38,18,0,191,240,46731,49.2,47.1,50.6
1,71003,2020,VIRGINIA,VA,loudoun,51107,US PRESIDENT,JOSEPH R BIDEN JR,DEMOCRAT,109335,...,31241,1149,77861,508,15236,26315,57513,36.5,36.2,36.8
2,71911,2020,VIRGINIA,VA,suffolkcity,51800,US PRESIDENT,JOSEPH R BIDEN JR,DEMOCRAT,20922,...,37869,153,1956,9,1003,4143,38613,38.1,37.1,39.0
3,70451,2020,VIRGINIA,VA,arlington,51013,US PRESIDENT,JOSEPH R BIDEN JR,DEMOCRAT,91963,...,21801,854,24468,288,12063,13710,73078,34.8,34.6,35.1
4,70509,2020,VIRGINIA,VA,bland,51021,US PRESIDENT,DONALD J TRUMP,REPUBLICAN,1925,...,250,0,13,0,11,93,23664,46.1,41.9,51.3


#MERGING DATAFRAMES INTO ONE:

This is where I cant get it to work. All of the column names are identical but it is raising an index error which I think has to do with the first values in the columns being the same but I guess this could be dropped somehow? idk

In [26]:
dfs = [merged_df_2008,
       merged_df_2012,
       merged_df_2016,
       merged_df_2020]
clean_df = pd.concat(dfs, ignore_index=True)
clean_df.head()

InvalidIndexError: Reindexing only valid with uniquely valued Index objects